<a href="https://colab.research.google.com/github/joudiabouseif/DLClus/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
 
 
uploaded = files.upload()

Saving df.csv to df.csv


In [2]:
import pandas as pd
#load the data
final=pd.read_csv("df.csv")

In [3]:
final['keywords'] = final['keywords'].str.lower()
  
final.head()

,datasetName,keywords
0,Football_Results,country league club player names matches playe...
1,Women_Football,date home team away team home score away score...
2,Int_Football,date home team away team home score away score...
3,games,game id year round date venue start time atten...
4,FMEL_Dataset,id season division round local team visitor te...


In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
final['keywords'] = final['keywords'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
final.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,datasetName,keywords
0,Football_Results,country league club player names matches playe...
1,Women_Football,date home team away team home score away score...
2,Int_Football,date home team away team home score away score...
3,games,game id year round date venue start time atten...
4,FMEL_Dataset,id season division round local team visitor te...


In [5]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

final['keywords'] = final['keywords'].apply(stem_sentences)

In [6]:
final.head()

,datasetName,keywords
0,Football_Results,countri leagu club player name match play subs...
1,Women_Football,date home team away team home score away score...
2,Int_Football,date home team away team home score away score...
3,games,game id year round date venu start time attend...
4,FMEL_Dataset,id season divis round local team visitor team ...


In [7]:
#tfidf vector initililization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(final['keywords'].values)
tfidf.shape

(25, 335)

In [8]:
from sklearn.cluster import KMeans
model_tf = KMeans(n_clusters=5, init='k-means++', n_init=7 , max_iter=300, random_state= 99)
model_tf.fit(tfidf)

KMeans(n_clusters=5, n_init=7, random_state=99)

In [9]:
labels_tf = model_tf.labels_
cluster_center_tf=model_tf.cluster_centers_

In [10]:
from sklearn import metrics
silhouette_score_tf = metrics.silhouette_score(tfidf, labels_tf, metric='euclidean')

In [11]:
# Giving Labels/assigning a cluster to each point/text 
df1 = final
df1['Tfidf Clus Label'] = model_tf.labels_
df1.head(25)

,datasetName,keywords,Tfidf Clus Label
0,Football_Results,countri leagu club player name match play subs...,4
1,Women_Football,date home team away team home score away score...,1
2,Int_Football,date home team away team home score away score...,1
3,games,game id year round date venu start time attend...,1
4,FMEL_Dataset,id season divis round local team visitor team ...,1
5,churn,row number custom id surnam credit score geogr...,3
6,telecom_churn,churn account week contract renew data plan da...,3
7,TelecoChurn,custom gender senior citizen partner depend te...,3
8,Churn_Modelling,row number custom id surnam credit score geogr...,3
9,cell2cellholdout,custom churn monthli revenu monthli minut tota...,3


In [12]:
# How many points belong to each cluster ->

df1.groupby(['Tfidf Clus Label'])['keywords'].count()

Tfidf Clus Label
0    5
1    4
2    4
3    5
4    7
Name: keywords, dtype: int64

In [13]:
silhouette_score_tf

0.15678922250065108

In [14]:
from sklearn.metrics.cluster import rand_score
results = df1['Tfidf Clus Label'].to_numpy()
actual = [0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4]
rand_score (actual,results)

0.9366666666666666